In [1]:
import os
import pathlib
import numpy as np
import tensorflow as tf
from   tensorflow.keras.layers.experimental import preprocessing
from   tensorflow.keras import layers
from   tensorflow.keras import models
from   tensorflow.keras.optimizers.schedules import PolynomialDecay


In [2]:
class SignalGenerator:
    def __init__(self, labels, sampling_rate, frame_length, frame_step,
            num_mel_bins=None, lower_frequency=None, upper_frequency=None,
            num_coefficients=None, mfcc=False):
        self.labels = labels
        self.sampling_rate = sampling_rate
        self.frame_length = frame_length
        self.frame_step = frame_step
        self.num_mel_bins = num_mel_bins
        self.lower_frequency = lower_frequency
        self.upper_frequency = upper_frequency
        self.num_coefficients = num_coefficients
        num_spectrogram_bins = (frame_length) // 2 + 1

        if mfcc is True:
            self.linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
                    self.num_mel_bins, num_spectrogram_bins, self.sampling_rate,
                    self.lower_frequency, self.upper_frequency)
            self.preprocess = self.preprocess_with_mfcc
        else:
            self.preprocess = self.preprocess_with_stft

    def read(self, file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        label = parts[-2]
        label_id = tf.argmax(label == self.labels)
        audio_binary = tf.io.read_file(file_path)
        audio, _ = tf.audio.decode_wav(audio_binary)
        audio = tf.squeeze(audio, axis=1)

        return audio, label_id

    def pad(self, audio):
        zero_padding = tf.zeros([self.sampling_rate] - tf.shape(audio), dtype=tf.float32)
        audio = tf.concat([audio, zero_padding], 0)
        audio.set_shape([self.sampling_rate])

        return audio

    def get_spectrogram(self, audio):
        stft = tf.signal.stft(audio, frame_length=self.frame_length,
                frame_step=self.frame_step, fft_length=self.frame_length)
        spectrogram = tf.abs(stft)

        return spectrogram

    def get_mfccs(self, spectrogram):
        mel_spectrogram = tf.tensordot(spectrogram,
                self.linear_to_mel_weight_matrix, 1)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
        mfccs = mfccs[..., :self.num_coefficients]

        return mfccs

    def preprocess_with_stft(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        spectrogram = tf.expand_dims(spectrogram, -1)
        spectrogram = tf.image.resize(spectrogram, [32, 32])

        return spectrogram, label

    def preprocess_with_mfcc(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        mfccs = self.get_mfccs(spectrogram)
        mfccs = tf.expand_dims(mfccs, -1)

        return mfccs, label

    def make_dataset(self, files, train):
        ds = tf.data.Dataset.from_tensor_slices(files)
        ds = ds.map(self.preprocess, num_parallel_calls=4)
        ds = ds.batch(32)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)
        return ds

In [8]:
def get_model(conv1, conv2, conv3, conv4):
    model = models.Sequential([
      layers.Input(shape=shape),
      layers.Conv2D(filters=conv1, kernel_size=[3,3], strides=[2,1], use_bias=False),
      layers.BatchNormalization(momentum=0.1),
      layers.ReLU(),
      layers.DepthwiseConv2D(kernel_size=[3,3], strides=[1,1], use_bias=False),
      layers.Conv2D(filters=conv2, kernel_size=[1,1], strides=[1,1], use_bias=False),
      layers.BatchNormalization(momentum=0.1),
      layers.ReLU(),
      layers.DepthwiseConv2D(kernel_size=[3,3], strides=[1,1], use_bias=False),
      layers.Conv2D(filters=conv3, kernel_size=[1,1], strides=[1,1], use_bias=False),
      layers.BatchNormalization(momentum=0.1),
      layers.ReLU(),
      layers.DepthwiseConv2D(kernel_size=[3,3], strides=[1,1], use_bias=False),
      layers.Conv2D(filters=conv4, kernel_size=[1,1], strides=[1,1], use_bias=False),
      layers.BatchNormalization(momentum=0.1),
      layers.ReLU(),
      layers.GlobalAveragePooling2D(),
      layers.Dense(8)
    ])
    return model 

In [4]:
!git clone https://github.com/xolotl18/ML_monitoring

Cloning into 'ML_monitoring'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [5]:
code_path = "ML_monitoring/data_splits"

data_dir = pathlib.Path('data/mini_speech_commands')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'mini_speech_commands.zip',
      origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
      extract=True,
      cache_dir='.', cache_subdir='data')
  
#lista di labels
labels=[]
for el in os.listdir("./data/mini_speech_commands"):
  if el!="README.md":
    labels.append(el)

#lista di training
training_list=[]
file=open(code_path+"/kws_train_split.txt")
for line in file:
  training_list.append('.'+line[1:-1])

#lista di validation
validation_list=[]
file=open(code_path+"/kws_val_split.txt")
for line in file:
  validation_list.append('.'+line[1:-1])

#lista di test
test_list=[]
file=open(code_path+"/kws_test_split.txt")
for line in file:
  test_list.append('.'+line[1:-1])

tot=len(training_list)+len(validation_list)+len(test_list)

182091776/182082353 [==============================] - 1s 0us/step


In [11]:
MFCC_OPTIONS = {'frame_length': 640, 'frame_step': 320, 'mfcc': True,'lower_frequency': 20, 'upper_frequency': 4000, 'num_mel_bins': 40,'num_coefficients': 10}
shape = [49, 10, 1]

model_options = {'conv1':64, 'conv2':96, 'conv3':128, 'conv4':256}

generator = SignalGenerator(labels, 16000, **MFCC_OPTIONS)
train_ds = generator.make_dataset(training_list, True)
val_ds = generator.make_dataset(validation_list, False)
test_ds = generator.make_dataset(test_list, False)

n_batches = 200

model = get_model(**model_options)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 24, 8, 64)         576       
                                                                 
 batch_normalization_8 (Batc  (None, 24, 8, 64)        256       
 hNormalization)                                                 
                                                                 
 re_lu_8 (ReLU)              (None, 24, 8, 64)         0         
                                                                 
 depthwise_conv2d_6 (Depthwi  (None, 22, 6, 64)        576       
 seConv2D)                                                       
                                                                 
 conv2d_9 (Conv2D)           (None, 22, 6, 96)         6144      
                                                                 
 batch_normalization_9 (Batc  (None, 22, 6, 96)       

In [12]:
learning_rate_fn = PolynomialDecay(
    initial_learning_rate=1e-3,
    decay_steps=3000,
    end_learning_rate=1e-5
    )

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['sparse_categorical_accuracy'] 
)

# callbacks
ckp_dir = "./checkpoint/"
try:
  os.mkdir(ckp_dir)
except FileExistsError:
  pass

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    ckp_dir, 
    monitor='val_sparse_categorical_accuracy', 
    verbose=0, 
    save_best_only=True,
    save_weights_only=False, 
    mode='max', 
    save_freq='epoch')

# fit model
EPOCHS = 30
history = model.fit(
    train_ds, 
    validation_data=val_ds,  
    epochs=EPOCHS,
    callbacks=[checkpoint_cb],
)

base_model = tf.keras.models.load_model(ckp_dir)
base_model.evaluate(test_ds, batch_size=32)

Epoch 1/30
200/200 [==============================] - 161s 414ms/step - loss: 1.1199 - sparse_categorical_accuracy: 0.6427 - val_loss: 0.7046 - val_sparse_categorical_accuracy: 0.7950
Epoch 2/30
200/200 [==============================] - 3s 14ms/step - loss: 0.4976 - sparse_categorical_accuracy: 0.8572 - val_loss: 0.4169 - val_sparse_categorical_accuracy: 0.8725
Epoch 3/30
200/200 [==============================] - 3s 13ms/step - loss: 0.3460 - sparse_categorical_accuracy: 0.8970 - val_loss: 0.3356 - val_sparse_categorical_accuracy: 0.9000
Epoch 4/30
200/200 [==============================] - 1s 6ms/step - loss: 0.2646 - sparse_categorical_accuracy: 0.9195 - val_loss: 0.2983 - val_sparse_categorical_accuracy: 0.8975
Epoch 5/30
200/200 [==============================] - 3s 14ms/step - loss: 0.2191 - sparse_categorical_accuracy: 0.9345 - val_loss: 0.2655 - val_sparse_categorical_accuracy: 0.9237
Epoch 6/30
200/200 [==============================] - 1s 6ms/step - loss: 0.1783 - sparse_cat

[0.2589355707168579, 0.9137499928474426]

In [27]:
base_model.save_weights('model_weights.h5')

In [28]:
model2 = get_model(**model_options)
model2.load_weights('model_weights.h5')

In [29]:
model2.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['sparse_categorical_accuracy'] 
)
model2.evaluate(test_ds, batch_size=32)

25/25 [==============================] - 0s 3ms/step - loss: 0.2589 - sparse_categorical_accuracy: 0.9137


[0.2589355707168579, 0.9137499928474426]